<br />
<div align="center">
  <a href="https://deepwok.github.io/">
    <img src="../imgs/deepwok.png" alt="Logo" width="160" height="160">
  </a>

  <h1 align="center">Lab 4 for Advanced Deep Learning Systems (ADLS) - Hardware Stream</h1>

  <p align="center">
    ELEC70109/EE9-AML3-10/EE9-AO25
    <br />
		Written by
    <a href="https://aaron-zhao123.github.io/">Aaron Zhao, Pedro Gimenes </a>
  </p>
</div>

# General introduction

In this lab, you will learn how to emit SystemVerilog code for a neural network that's been transformed and optimized by MASE. Then, you'll design some hardware for a new Pytorch layer, and simulate the hardware using your new module.

# The Hardware Emit pass

The `emit_verilog` transform pass generates a top-level RTL file and testbench file according to the `MaseGraph`, which includes a hardware implementation of each layer in the network. This top-level file instantiates modules from the `components` library in MASE and/or modules generated using [HLS](https://en.wikipedia.org/wiki/High-level_synthesis), when internal components are not available. The hardware can then be simulated using [Verilator](https://www.veripool.org/verilator/), or deployed on an FPGA.

First, add Machop to your system PATH (if you haven't already done so) and import the required libraries.

In [ ]:
import os, sys

os.path.abspath("")

sys.path.append(
    os.path.join(
        os.path.abspath(""),
        "..",
        "..",
        "machop",
    )
)

from chop.ir.graph.mase_graph import MaseGraph

from chop.passes.graph.analysis import (
    init_metadata_analysis_pass,
    add_common_metadata_analysis_pass,
    add_hardware_metadata_analysis_pass,
    report_node_type_analysis_pass,
)

from chop.passes.graph.transforms import (
    emit_verilog_top_transform_pass,
    emit_internal_rtl_transform_pass,
    emit_bram_transform_pass,
    emit_verilog_tb_transform_pass,
    quantize_transform_pass,
)

from chop.tools.logger import set_logging_verbosity

set_logging_verbosity("debug")

import toml
import torch
import torch.nn as nn

Now, define the neural network. We're using a model which can be used to perform digit classification on the MNIST dataset.

In [ ]:
class MLP(torch.nn.Module):
    """
    Toy FC model for digit recognition on MNIST
    """

    def __init__(self) -> None:
        super().__init__()

        self.fc1 = nn.Linear(28 * 28, 28 * 28)
        self.fc2 = nn.Linear(28 * 28, 28 * 28 * 4)
        self.fc3 = nn.Linear(28 * 28 * 4, 10)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Now, we'll generate a MaseGraph and add metadata. At this point, it's important to run the `add_hardware_metadata` analysis pass. This adds all the required metadata which is later used by the `emit_verilog` pass, including:

1. The node's toolchain, which defines whether we use internal Verilog modules from the `components` library or the HLS flow.
2. The Verilog parameters associated with each node.

> **_TASK:_** Read [this page](https://jianyicheng.github.io/mase-tools/modules/analysis/add_metadata.html#add-hardware-metadata-analysis-pass) for more information on the hardware metadata pass.


In [ ]:
mlp = MLP()
mg = MaseGraph(model=mlp)

# Provide a dummy input for the graph so it can use for tracing
batch_size = 1
x = torch.randn((batch_size, 28, 28))
dummy_in = {"x": x}

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(
    mg, {"dummy_in": dummy_in, "add_value": False}
)
mg, _ = add_hardware_metadata_analysis_pass(mg, None)

Before running `emit_verilog`, we'll quantize the model to fixed precision. Refer back to [lab 2](https://github.com/DeepWok/mase/blob/main/docs/labs/lab2.md) if you've forgotten how this works. Check that the data type for each node is correct after quantization.

In [ ]:
config_file = os.path.join(
    os.path.abspath(""),
    "..",
    "..",
    "machop",
    "configs",
    "tests",
    "quantize",
    "fixed.toml",
)
with open(config_file, "r") as f:
    quan_args = toml.load(f)["passes"]["quantize"]
mg, _ = quantize_transform_pass(mg, quan_args)

_ = report_node_type_analysis_pass(mg)

# Update the metadata
for node in mg.fx_graph.nodes:
    for arg, arg_info in node.meta["mase"].parameters["common"]["args"].items():
        if isinstance(arg_info, dict):
            arg_info["type"] = "fixed"
            arg_info["precision"] = [8, 3]
    for result, result_info in (
        node.meta["mase"].parameters["common"]["results"].items()
    ):
        if isinstance(result_info, dict):
            result_info["type"] = "fixed"
            result_info["precision"] = [8, 3]

Finally, run the emit verilog pass to generate the SystemVerilog files.

In [ ]:
mg, _ = emit_verilog_top_transform_pass(mg)
mg, _ = emit_internal_rtl_transform_pass(mg)

cosim_config = {"test_inputs": [x], "trans_num": 1}

The generated files should now be found under `top/hardware`. 

> **_TASK:_** Read through `top/hardware/rtl/top.sv` and make sure you understand how our MLP model maps to this hardware design. 

You will notice the following instantiated modules:

* `fixed_linear`: this is found under `components/linear/fixed_linear.sv` and implements each Linear layer in the model.
* `fc<layer number>_weight/bias_source`: these are [BRAM](https://nandland.com/lesson-15-what-is-a-block-ram-bram/) memories which drive the weights and biases into the linear layers for computation.
* `fixed_relu`: found under `components/activations/fixed_relu.sv`, implements the ReLU activation.

As of now, we can't yet run a simulation on the model, as we haven't yet generated the memory components. To do this, run the `emit_bram` transform pass as follows, which will generate the memory initialization files and SystemVerilog modules to drive weights and biases into the linear layers. Finally, the `emit_verilog_tb` transform pass will generate the testbench files.


In [ ]:
mg, _ = emit_bram_transform_pass(mg)
mg, _ = emit_verilog_tb_transform_pass(mg, pass_args=cosim_config)

> **_TASK:_** Launch a simulation as follows.

In [ ]:
!chmod +x top/hardware/sim/build.sh
!top/hardware/sim/build.sh

# Extension Task

Pytorch has a number of layers which are available to users to define neural network models. At the moment, `emit_verilog` supports generating Verilog for models including Linear layers and the ReLU activation.

> **_EXTENSION TASK:_** choose another layer type from the [Pytorch list](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity) and write a SystemVerilog file to implement that layer in hardware. Then, change the generated `top.sv` file to inject that layer within the design. For example, you may replace the ReLU activations with [Leaky ReLU](https://pytorch.org/docs/stable/generated/torch.nn.RReLU.html#torch.nn.RReLU). Re-build the simulation and observe the effect on latency and accuracy.